# Web Scrapping with API

In [1]:
import config
import requests

In [2]:
# Create a list to store comment dictionaries
comments_list = []

In [3]:
company = ""
with open("company.txt","r") as file:
    company = file.read()

# Access Database

In [4]:
#!pip install pymysql

In [5]:
import connection

In [6]:
con = connection.connect()
stm = con.cursor()

In [7]:
q = f"select * from companydetails where CompanyName='{company}'"
stm.execute(q)

1

In [8]:
rows = stm.fetchall()

In [9]:
rows

(('Company 2',
  'company.name.2.test@gmail.com',
  '61553785493448',
  '122094113372126183',
  '17993439215366446',
  'EAALZCZCJv92uABOylJzf63QDrpEQu90mX0AeroL4XaZAl9WQ0tBuGDiiGZCAEc70umtwXkKilqcGIKJ24hS3uHfhGbjLUXnSuA0xXXRit8kdnfjinFzyrHreni8Jzxh6NmgjgGRBIZACuq5q7cYBleFo3I23sP6CDnZA5DDYWtohlwuKeYGFfND6LlXispc2id',
  'EAALZCZCJv92uABOylJzf63QDrpEQu90mX0AeroL4XaZAl9WQ0tBuGDiiGZCAEc70umtwXkKilqcGIKJ24hS3uHfhGbjLUXnSuA0xXXRit8kdnfjinFzyrHreni8Jzxh6NmgjgGRBIZACuq5q7cYBleFo3I23sP6CDnZA5DDYWtohlwuKeYGFfND6LlXispc2id',
  11.886492908000946),)

In [10]:
Reciver_Mail = rows[0][1]
page_id = rows[0][2]
FB_post_id = rows[0][3]
INS_post_id = rows[0][4]
FB_access = rows[0][5]
INS_access = rows[0][6]
prev = rows[0][7]

## (a) Facebook 
`Meta Graph API`

In [11]:
#Scrapping Facebook Comments
user_access_token = FB_access

url = f'https://graph.facebook.com/v18.0/{page_id}_{FB_post_id}/comments'

params = {
    'access_token': user_access_token,
    'fields': 'message,created_time',
}

response = requests.get(url, params=params)
data = response.json()
#print(data)
# Populate the list with comments
if 'data' in data:
    for comment in data['data']:
        from_info = comment.get('from', {})
        comments_list.append({
                                    'Message': comment['message'],
                                    'Timestamp': comment['created_time'],
                                    'Source' : 'Facebook',
                              })
for comment in comments_list:
    print(comment)

## (b) Instagram 
`Meta Graph API`

In [12]:
#Scrapping Instagram comments
user_access_token = INS_access
url = f'https://graph.facebook.com/v18.0/{INS_post_id}/comments'
params = {
    'fields': 'text,timestamp',
    'access_token': user_access_token,
}

response = requests.get(url, params=params)
data = response.json()
#print(data)
# Populate the list with comments
if 'data' in data:
    for comment in data['data']:
        from_info = comment.get('from', {})
        comments_list.append({
                                    'Message':comment['text'],
                                    'Timestamp': comment['timestamp'],
                                    'Source' : 'Instagram',
                            })

for comment in comments_list:
    print(comment)

{'Message': 'its not good', 'Timestamp': '2023-11-16T16:46:33+0000', 'Source': 'Instagram'}
{'Message': 'its beautiful', 'Timestamp': '2023-11-16T14:24:28+0000', 'Source': 'Instagram'}
{'Message': 'nice pic', 'Timestamp': '2023-11-16T14:18:17+0000', 'Source': 'Instagram'}


## Sentiment Analysis

## Roberta Pretrained Model

In [13]:
#!pip install matplotlib

In [14]:
#!pip install seaborn

In [15]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [16]:
#!pip install tqdm

In [17]:
from tqdm.notebook import tqdm

In [18]:
#pip install pywinpty

In [19]:
#!pip install transformers

In [20]:
#!pip install scipy

In [21]:
from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification
from scipy.special import softmax

In [22]:
df = pd.DataFrame(comments_list, columns=['Message','Timestamp','Source'])

In [23]:
print(df.dtypes)
df

Message      object
Timestamp    object
Source       object
dtype: object


,Message,Timestamp,Source
0,its not good,2023-11-16T16:46:33+0000,Instagram
1,its beautiful,2023-11-16T14:24:28+0000,Instagram
2,nice pic,2023-11-16T14:18:17+0000,Instagram


In [24]:
df['Timestamp'] = pd.to_datetime(df['Timestamp'])

In [25]:
print(df.dtypes)

Message                   object
Timestamp    datetime64[ns, UTC]
Source                    object
dtype: object


In [26]:
df.sort_values(by='Timestamp', inplace = True)

In [27]:
df

,Message,Timestamp,Source
2,nice pic,2023-11-16 14:18:17+00:00,Instagram
1,its beautiful,2023-11-16 14:24:28+00:00,Instagram
0,its not good,2023-11-16 16:46:33+00:00,Instagram


In [28]:
MODEL = f"cardiffnlp/twitter-roberta-base-sentiment"
tokenizer = AutoTokenizer.from_pretrained(MODEL)
model = AutoModelForSequenceClassification.from_pretrained(MODEL)

In [29]:
def polarity_scores_roberta(example):
    encoded_text = tokenizer(example, return_tensors='pt')
    output = model(**encoded_text)
    scores = output[0][0].detach().numpy()
    scores = softmax(scores)
    scores_dict = {
        'roberta_neg' : scores[0],
        'roberta_neu' : scores[1],
        'roberta_pos' : scores[2]
    }
    return scores_dict

In [30]:
#!pip install --upgrade jupyter

In [31]:
#!pip install --upgrade ipywidgets

In [32]:
base : float
base = 10.00

res = {
    'pos' : [],
    'neg' : []
}
for i,row in tqdm(df.iterrows(), total=len(df)):
    try:
        print(row);
        text = row['Message']
        #myid = row['Id']
        roberta_result = polarity_scores_roberta(text)
        print(roberta_result)
        if roberta_result['roberta_neg']>roberta_result['roberta_neu'] and roberta_result['roberta_neg']>roberta_result['roberta_pos'] :
            base-=roberta_result['roberta_neg']
            res['neg'].append({
                'Comment' : row['Message'],
                'Source' :row['Source']
            })
        if roberta_result['roberta_pos']>roberta_result['roberta_neu'] and roberta_result['roberta_pos']>roberta_result['roberta_neg'] :
            base+=roberta_result['roberta_pos']
            res['pos'].append({
                'Comment' : row['Message'],
                'Source' :row['Source']
            })
        #print(base);
    except RuntimeError:
        print(f'Broke for id {i}')

  0%|          | 0/3 [00:00<?, ?it/s]

Message                       nice pic
Timestamp    2023-11-16 14:18:17+00:00
Source                       Instagram
Name: 2, dtype: object
{'roberta_neg': 0.0043418566, 'roberta_neu': 0.055478096, 'roberta_pos': 0.94018006}
Message                  its beautiful
Timestamp    2023-11-16 14:24:28+00:00
Source                       Instagram
Name: 1, dtype: object
{'roberta_neg': 0.0060792877, 'roberta_neu': 0.04760788, 'roberta_pos': 0.94631284}
Message                   its not good
Timestamp    2023-11-16 16:46:33+00:00
Source                       Instagram
Name: 0, dtype: object
{'roberta_neg': 0.94172156, 'roberta_neu': 0.049522933, 'roberta_pos': 0.008755575}


In [33]:
print(base)

10.944771349430084


# Update Database

In [34]:
q = f"update companydetails set Score = {base} where CompanyName = '{company}'"
stm.execute(q)
con.commit()

# Sending Email

In [35]:
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart

In [36]:
def send_email(subject, body, to_email, smtp_server, smtp_port, smtp_username, smtp_password):
    # Create the MIME object
    msg = MIMEMultipart()
    msg['From'] = smtp_username
    msg['To'] = to_email
    msg['Subject'] = subject

    msg.attach(MIMEText(body, 'plain'))

    server = smtplib.SMTP(smtp_server, smtp_port)
    server.starttls()
    server.login(smtp_username, smtp_password)

    server.sendmail(smtp_username, to_email, msg.as_string())
    server.quit()

In [37]:
subject = "Change In Score Detected"
body = f"New Score : {base} \nDelta change : {base-prev} \nLast 5 negitive comments:{res['neg'][-5:]} \nLast 5 positive comments:{res['pos'][-5:]}"

In [38]:
smtp_server = "smtp.gmail.com"
smtp_port = 587 

In [39]:
if(prev!=base) :
    send_email(subject=subject,
            body=body,
            to_email=Reciver_Mail,
            smtp_server=smtp_server,
            smtp_port=smtp_port,
            smtp_username=config.Sender_mail,
            smtp_password=config.AppPassword)
    print("Mail send")

Mail send


In [40]:
stm.close()
con.close()

In [41]:
text = f"{prev} {base}"

In [42]:
with open('score.txt','w') as file:
    file.write(text)